In [11]:
import argparse
from Bio import SeqIO, Seq
import random
import numpy as np
import pysam
from simulateCNVutil import *

In [2]:
prob=[0.1,0.01,0.001]

In [28]:
def mutate(mutable_seq,prob):
    ct=0
    length = len(mutable_seq)
    NucList = {'A':['C','G','T'],'C':['A','G','T'],'G':['A','C','T'],'T':['A','C','G']}
    vals=np.random.binomial(size=length,n=1,p=prob)
    changes=[]
    for i, nt in enumerate(mutable_seq):
        if vals[i]:
            nt=nt.upper()
            sub=random.choice(NucList[nt])
            mutable_seq = "".join( [ "".join(mutable_seq.split()[:i]) , sub , "".join(mutable_seq.split()[i+1:])])
            ct=ct+1
            changes.append((i,sub))
    print(length, ct)
    return mutable_seq.upper(),changes

In [26]:
def fastify(seq, chrom,start,end):
    return "\n".join(     [ "/".join([">"+chrom,str(start),str(end)])   , seq    ]   )

In [5]:
def invert(mutable_seq):
    return mutable_seq[::-1].upper()

In [6]:
#s at start
#l #r, left third right third
#m middle
#e at end

def insert(mutable_seq, mutated_seq, pos):
    length = len(mutable_seq)
    convert={'s':0,'e': length-1, 'l': round(length/3), 'r':round(2*length/3),'m':round(length/2)  }
    if type(pos)==str:
        poss = convert[pos]
    elif type(pos)==int:
        poss=pos
    seq = "".join( [ "".join(mutable_seq[:poss]) , mutated_seq , "".join(mutable_seq[poss:])])
    return seq.upper()

In [16]:
#chrom='chr6'
#start=160609100
#end=160648600
chrom='chr4'	
start=70314800	
end=70326600

In [17]:
ref = pysam.FastaFile('/home/cmb-16/mjc/shared/references/hg38/hg38.fa')
pre=ref.fetch(chrom, start-1, end-1)

In [18]:
name = "/".join([chrom,str(start),str(end)])


In [12]:
Seqq=Mutable_seq(pre,name)
Seqq.mutate(0.001)

NameError: name 'np' is not defined

In [31]:
mPre,mPre_changes=mutate(pre,0.01)

11800 152


In [32]:
new_pre=insert(pre, mPre, 'e')

In [33]:
new_pre=new_pre.upper()

In [35]:
with open('simulateCNV/mut_seq.fasta','w') as mut:
    mut.write(fastify(new_pre, chrom,start,end))

In [36]:
mid_pre = insert(pre,mPre,'m')

In [37]:
mid_pre=mid_pre.upper()

In [38]:
with open('simulateCNV/mut_seq.mid.fasta','w') as mut:
    mut.write(fastify(mid_pre, chrom,start,end))

In [39]:
mPre_changes[0]

(204, 'A')